# Conjunto de dados de estatísticas de saúde cardíaca


Esse conjunto de dados inclui informações demográficas, sinais vitais e indicadores do estado de saúde do coração dos indivíduos. Inclui características como idade, sexo, pressão arterial, níveis de colesterol e se o indivíduo sofreu um ataque cardíaco. O conjunto de dados é valioso para analisar tendências relacionadas à saúde cardiovascular e avaliar os fatores de risco associados a doenças cardíacas. Pesquisadores e profissionais de saúde podem utilizar esses dados para identificar padrões, desenvolver modelos preditivos e formular intervenções personalizadas para prevenção e manejo de doenças cardíacas.

Link: https://www.kaggle.com/datasets/mahad049/heart-health-stats-dataset

In [ ]:
import pandas as pd

dados = pd.read_csv('/content/Heart_health.csv', sep = ',')
dados.head()

,ID,Name,Age,Gender,Height(cm),Weight(kg),Blood Pressure(mmHg),Cholesterol(mg/dL),Glucose(mg/dL),Smoker,Exercise(hours/week),Heart Attack
0,1,John Doe,45,Male,175,80,120/80,200,90,No,3,0
1,2,Jane Smith,35,Female,160,65,110/70,180,80,No,2,0
2,3,Michael Johnson,55,Male,180,85,130/85,220,95,Yes,4,1
3,4,Sarah Brown,40,Female,165,70,115/75,190,85,No,3,0
4,5,David Lee,50,Male,170,75,125/80,210,92,Yes,2,1


In [ ]:
dados.drop(columns = ['ID','Name'], inplace = True)
dados.head()

In [ ]:
# Vamos criar a PAM

dados['PAS'],dados['PAD'] = dados['Blood Pressure(mmHg)'].str.split('/',1).str
dados

In [ ]:
dados.drop('Blood Pressure(mmHg)', axis=1, inplace=True)
dados.head()

In [ ]:
dados.info()

In [ ]:
dados = dados.astype({'PAS':'int64', 'PAD':'int64'})

In [ ]:
dados.info()

In [ ]:
dados['PAM'] = (dados['PAS'] + 2 * dados['PAD'])/3
dados

In [ ]:
dados.drop(columns = ['PAS','PAD'], inplace = True)
dados

In [ ]:
dados = pd.get_dummies(data= dados, columns=['Gender','Smoker','Heart Attack'], drop_first = True)
dados.head()

In [ ]:
# Renomeando as colunas usando um dicionário
dados = dados.rename(columns={'Age': 'Idade', 'Height(cm)': 'Altura',
                        'Weight(kg)' : 'Peso',
                        'Cholesterol(mg/dL)' : 'Colesterol',
                        'Glucose(mg/dL)':'Glicose',
                        'Exercise(hours/week)': "Exercicio",
                        'Gender_Male':'Genero',
                        'Smoker_Yes':'Fuma',
                        'Heart Attack_1':'Target'})

# Exibindo o DataFrame com os nomes das colunas atualizados
dados.head()


In [ ]:
import seaborn as sns # biblioteca de visualização de informações estatísticas
import matplotlib.pyplot as plt # biblioteca de visualização de dados
import statsmodels.api as sm # biblioteca de modelagem estatística
import numpy as np # biblioteca para operações matemáticas multidimensionais
from scipy import stats # estatística chi2
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos
import plotly.graph_objs as go # gráfico 3D
import statsmodels.formula.api as smf # estimação do modelo logístico binário

import warnings

In [ ]:
# In[ ]: Plotando a curva sigmóide teórica de ocorrência de um evento para um
#range do logito z entre -5 e +5

from math import exp

#Estabelecendo uma função para a probabilidade de ocorrência de um evento
def prob(z):
    return 1 / (1 + exp(-z))

logitos = []
probs = []

for i in np.arange(-5,6):
    logitos.append(i)
    probs.append(prob(i))

df = pd.DataFrame({'logito':logitos,'probs':probs})

plt.figure(figsize=(10,10))
plt.plot(df.logito, df.probs, color='darkorchid')
plt.scatter(df.logito, df.probs, color = 'darkorchid', s=100)
plt.axhline(y = df.probs.mean(), color = 'grey', linestyle = ':')
plt.xlabel("Logito Z", fontsize=20)
plt.ylabel("Probabilidade", fontsize=20)
plt.show()

In [ ]:
lista_colunas = list(dados.drop(columns=['Target']).columns)

formula_dummies_modelo = ' + '.join(lista_colunas)

formula_dummies_modelo = "Target ~ " + formula_dummies_modelo

print("Fórmula utilizada: ",formula_dummies_modelo)

#Modelo propriamente dito
modelo = sm.Logit.from_formula(formula_dummies_modelo,
                                               dados).fit()

#Parâmetros do modelo
modelo.summary()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,\
    ConfusionMatrixDisplay, recall_score

def matriz_confusao(predicts, observado, cutoff):

    values = predicts.values

    predicao_binaria = []

    for item in values:
        if item < cutoff:
            predicao_binaria.append(0)
        else:
            predicao_binaria.append(1)

    cm = confusion_matrix(predicao_binaria, observado)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.xlabel('True')
    plt.ylabel('Classified')
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    plt.show()

    sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
    especificidade = recall_score(observado, predicao_binaria, pos_label=0)
    acuracia = accuracy_score(observado, predicao_binaria)

    #Visualizando os principais indicadores desta matriz de confusão
    indicadores = pd.DataFrame({'Sensitividade':[sensitividade],
                                'Especificidade':[especificidade],
                                'Acurácia':[acuracia]})
    return indicadores

In [ ]:
# Adicionando os valores previstos de probabilidade na base de dados
dados['phat'] = modelo.predict()

#Matriz de confusão para cutoff = 0.5
matriz_confusao(observado=dados['Target'],
                predicts=dados['phat'],
                cutoff=0.50)

In [ ]:
def espec_sens(observado,predicts):

    # adicionar objeto com os valores dos predicts
    values = predicts.values

    # range dos cutoffs a serem analisados em steps de 0.01
    cutoffs = np.arange(0,1.01,0.01)

    # Listas que receberão os resultados de especificidade e sensitividade
    lista_sensitividade = []
    lista_especificidade = []

    for cutoff in cutoffs:

        predicao_binaria = []

        # Definindo resultado binário de acordo com o predict
        for item in values:
            if item >= cutoff:
                predicao_binaria.append(1)
            else:
                predicao_binaria.append(0)

        # Cálculo da sensitividade e especificidade no cutoff
        sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
        especificidadee = recall_score(observado, predicao_binaria, pos_label=0)

        # Adicionar valores nas listas
        lista_sensitividade.append(sensitividade)
        lista_especificidade.append(especificidadee)

    # Criar dataframe com os resultados nos seus respectivos cutoffs
    resultado = pd.DataFrame({'cutoffs':cutoffs,'sensitividade':lista_sensitividade,'especificidade':lista_especificidade})
    return resultado

In [ ]:
dados_plotagem = espec_sens(observado = dados['Target'],
                            predicts = dados['phat'])
dados_plotagem

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dados_plotagem.cutoffs,dados_plotagem.sensitividade, '-o',
         color='indigo')
plt.plot(dados_plotagem.cutoffs,dados_plotagem.especificidade, '-o',
         color='limegreen')
plt.legend(['Sensitividade', 'Especificidade'], fontsize=17)
plt.xlabel('Cuttoff', fontsize=14)
plt.ylabel('Sensitividade / Especificidade', fontsize=14)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

#Função 'roc_curve' do pacote 'metrics' do sklearn

fpr, tpr, thresholds =roc_curve(dados['Target'],
                                dados['phat'])
roc_auc = auc(fpr, tpr)

#Cálculo do coeficiente de GINI
gini = (roc_auc - 0.5)/(0.5)

#Plotando a curva ROC
plt.figure(figsize=(10,10))
plt.plot(fpr,tpr, '-o', color='red')
plt.plot(fpr,fpr, ':', color='gray')
plt.title('Área abaixo da curva: %g' % round(roc_auc,4) +
          ' | Coeficiente de GINI: %g' % round(gini,4), fontsize=17)
plt.xlabel('1 - Especificidade', fontsize=15)
plt.ylabel('Sensitividade', fontsize=15)
plt.show()

In [ ]:
!pip install pandas-profiling

In [ ]:
from ydata_profiling import ProfileReport ## https://docs.profiling.ydata.ai/latest/

ProfileReport(dados)

In [ ]:
!pip install factor_analyzer

In [ ]:
!pip install pingouin

In [ ]:
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
import pingouin as pg
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dados.drop('phat', axis =1, inplace=true)


In [ ]:
#%% Teste de Bartlett

bartlett, p_value = calculate_bartlett_sphericity(dados)

print(f'Bartlett statistic: {bartlett}')

print(f'p-value : {p_value}')

In [ ]:
#%% Estatística KMO

kmo_all, kmo_model = calculate_kmo(dados)

print(f'kmo_model : {kmo_model}')

In [ ]:
fa = FactorAnalyzer()
fa.fit(dados)

In [ ]:

ev, v = fa.get_eigenvalues()

print(ev)

In [ ]:
fa.set_params(n_factors = 2, method = 'principal', rotation = None)
fa.fit(dados)


In [ ]:
#%% Eigenvalues, variâncias e variâncias acumulada

eigen_fatores = fa.get_factor_variance()
eigen_fatores

tabela_eigen = pd.DataFrame(eigen_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','Variância', 'Variância Acumulada']
tabela_eigen = tabela_eigen.T

print(tabela_eigen)

In [ ]:
cargas_fatores = fa.loadings_

tabela_cargas = pd.DataFrame(cargas_fatores)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = dados.columns
tabela_cargas

print(tabela_cargas)

In [ ]:
#%% Determinando as comunalidades

comunalidades = fa.get_communalities()

tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = dados.columns
tabela_comunalidades

print(tabela_comunalidades)

In [ ]:
#%% Resultados dos fatores para as observações do dataset (predict)

predict_fatores= pd.DataFrame(fa.transform(dados))
predict_fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(predict_fatores.columns)]

print(predict_fatores)

# Adicionando ao dataset

dados = pd.concat([dados.reset_index(drop=True), predict_fatores], axis=1)

dados

In [ ]:

corr_fator = pg.rcorr(dados[['Fator 1','Fator 2']], method = 'pearson', upper = 'pval', decimals = 4, pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})
print(corr_fator)

In [ ]:
# Renomeando as colunas usando um dicionário
dados = dados.rename(columns={'Fator 1': 'Fator1', 'Fator 2': 'Fator2'})

# Exibindo o DataFrame com os nomes das colunas atualizados
dados.head()

In [ ]:
colunas = ['Fator1','Fator2','Target']
dados2 = dados[colunas].copy()
dados2

In [ ]:
lista_colunas = list(dados2.drop(columns=['Target']).columns)

formula_dummies_modelo = ' + '.join(lista_colunas)

formula_dummies_modelo = "Target ~ " + formula_dummies_modelo

print("Fórmula utilizada: ",formula_dummies_modelo)

#Modelo propriamente dito
modelo = sm.Logit.from_formula(formula_dummies_modelo,
                                               dados2).fit()

#Parâmetros do modelo
modelo.summary()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,\
    ConfusionMatrixDisplay, recall_score

def matriz_confusao(predicts, observado, cutoff):

    values = predicts.values

    predicao_binaria = []

    for item in values:
        if item < cutoff:
            predicao_binaria.append(0)
        else:
            predicao_binaria.append(1)

    cm = confusion_matrix(predicao_binaria, observado)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.xlabel('True')
    plt.ylabel('Classified')
    plt.gca().invert_xaxis()
    plt.gca().invert_yaxis()
    plt.show()

    sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
    especificidade = recall_score(observado, predicao_binaria, pos_label=0)
    acuracia = accuracy_score(observado, predicao_binaria)

    #Visualizando os principais indicadores desta matriz de confusão
    indicadores = pd.DataFrame({'Sensitividade':[sensitividade],
                                'Especificidade':[especificidade],
                                'Acurácia':[acuracia]})
    return indicadores

In [ ]:
# Adicionando os valores previstos de probabilidade na base de dados
dados['phat'] = modelo.predict()

#Matriz de confusão para cutoff = 0.5
matriz_confusao(observado=dados['Target'],
                predicts=dados['phat'],
                cutoff=0.50)

In [ ]:
def espec_sens(observado,predicts):

    # adicionar objeto com os valores dos predicts
    values = predicts.values

    # range dos cutoffs a serem analisados em steps de 0.01
    cutoffs = np.arange(0,1.01,0.01)

    # Listas que receberão os resultados de especificidade e sensitividade
    lista_sensitividade = []
    lista_especificidade = []

    for cutoff in cutoffs:

        predicao_binaria = []

        # Definindo resultado binário de acordo com o predict
        for item in values:
            if item >= cutoff:
                predicao_binaria.append(1)
            else:
                predicao_binaria.append(0)

        # Cálculo da sensitividade e especificidade no cutoff
        sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
        especificidadee = recall_score(observado, predicao_binaria, pos_label=0)

        # Adicionar valores nas listas
        lista_sensitividade.append(sensitividade)
        lista_especificidade.append(especificidadee)

    # Criar dataframe com os resultados nos seus respectivos cutoffs
    resultado = pd.DataFrame({'cutoffs':cutoffs,'sensitividade':lista_sensitividade,'especificidade':lista_especificidade})
    return resultado

In [ ]:
dados_plotagem = espec_sens(observado = dados['Target'],
                            predicts = dados['phat'])
dados_plotagem

In [ ]:
def espec_sens(observado,predicts):

    # adicionar objeto com os valores dos predicts
    values = predicts.values

    # range dos cutoffs a serem analisados em steps de 0.01
    cutoffs = np.arange(0,1.01,0.01)

    # Listas que receberão os resultados de especificidade e sensitividade
    lista_sensitividade = []
    lista_especificidade = []

    for cutoff in cutoffs:

        predicao_binaria = []

        # Definindo resultado binário de acordo com o predict
        for item in values:
            if item >= cutoff:
                predicao_binaria.append(1)
            else:
                predicao_binaria.append(0)

        # Cálculo da sensitividade e especificidade no cutoff
        sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
        especificidadee = recall_score(observado, predicao_binaria, pos_label=0)

        # Adicionar valores nas listas
        lista_sensitividade.append(sensitividade)
        lista_especificidade.append(especificidadee)

    # Criar dataframe com os resultados nos seus respectivos cutoffs
    resultado = pd.DataFrame({'cutoffs':cutoffs,'sensitividade':lista_sensitividade,'especificidade':lista_especificidade})
    return resultado

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dados_plotagem.cutoffs,dados_plotagem.sensitividade, '-o',
         color='indigo')
plt.plot(dados_plotagem.cutoffs,dados_plotagem.especificidade, '-o',
         color='limegreen')
plt.legend(['Sensitividade', 'Especificidade'], fontsize=17)
plt.xlabel('Cuttoff', fontsize=14)
plt.ylabel('Sensitividade / Especificidade', fontsize=14)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

#Função 'roc_curve' do pacote 'metrics' do sklearn

fpr, tpr, thresholds =roc_curve(dados['Target'],
                                dados['phat'])
roc_auc = auc(fpr, tpr)

#Cálculo do coeficiente de GINI
gini = (roc_auc - 0.5)/(0.5)

#Plotando a curva ROC
plt.figure(figsize=(10,10))
plt.plot(fpr,tpr, '-o', color='red')
plt.plot(fpr,fpr, ':', color='gray')
plt.title('Área abaixo da curva: %g' % round(roc_auc,4) +
          ' | Coeficiente de GINI: %g' % round(gini,4), fontsize=17)
plt.xlabel('1 - Especificidade', fontsize=15)
plt.ylabel('Sensitividade', fontsize=15)
plt.show()